# Routing Design Pattern: IT Support Agent

## Overview
This notebook demonstrates the **Routing** pattern - a workflow where user queries are analyzed for intent, then routed to specialized external services based on classification. Unlike prompt chaining with predetermined steps, routing dynamically directs requests to different service handlers.

## Use Case
An IT Support chatbot that:
1. Extracts user intent from support queries
2. Routes to appropriate external service (password reset, hardware, software, network)
3. Retrieves service-specific response data
4. Generates personalized, actionable responses with verification steps

## Architecture
```
User Query → Extract Intent → Route Based on Intent → [External Services] → Generate Response
                                        ├─ Password Reset Service
                                        ├─ Hardware Issue Service
                                        ├─ Software Issue Service
                                        └─ Network Problem Service
```

## 1. Setup & Dependencies

Install required packages for building the routing workflow with LangGraph.

In [ ]:
!pip install -q langgraph langchain-core langchain-openai

## 2. Import Libraries

Import necessary modules for state management, message handling, LLM interactions, and JSON data loading.

In [ ]:
import os
import json
from typing import Dict, Optional, List, TypedDict
import warnings
warnings.filterwarnings('ignore')

# LangGraph imports
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# LangChain OpenAI
from langchain_openai import ChatOpenAI

print("✅ Dependencies loaded successfully")

## 3. Environment Configuration

Configure OpenAI API credentials. For Google Colab, use userdata to securely store your API key.

In [ ]:
# For Google Colab - Load API key from userdata
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    print("✅ API key loaded from Google Colab userdata")
except ImportError:
    # For local environment - set your API key directly
    os.environ["OPENAI_API_KEY"] = "your-api-key-here"
    print("⚠️  Running locally - Please set your API key")

# Initialize ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.2,
    api_key=os.environ.get("OPENAI_API_KEY")
)

print("✅ ChatOpenAI model configured")
print(f"   Model: {llm.model_name}")
print(f"   Temperature: {llm.temperature}")

## 4. Load External Service Data

Load mock external service responses from JSON files. These simulate real API calls to password reset, hardware, software, and network support systems.

In [ ]:
def load_external_service_data():
    """Load all external service response data from JSON files"""
    data_dir = 'data'
    
    with open(f'{data_dir}/password_reset_responses.json', 'r') as f:
        password_data = json.load(f)
    
    with open(f'{data_dir}/hardware_issue_responses.json', 'r') as f:
        hardware_data = json.load(f)
    
    with open(f'{data_dir}/software_issue_responses.json', 'r') as f:
        software_data = json.load(f)
    
    with open(f'{data_dir}/network_problem_responses.json', 'r') as f:
        network_data = json.load(f)
    
    return {
        'password_reset': password_data[0],
        'hardware_issue': hardware_data[0],
        'software_issue': software_data[0],
        'network_problem': network_data[0]
    }

# Load the external service data
SERVICE_DATA = load_external_service_data()

print("✅ External service data loaded")
print(f"📊 Available services: {list(SERVICE_DATA.keys())}")

## 5. State Definition

Define the state schema that flows through the routing workflow. Each node reads from and updates this shared state.

In [ ]:
class ITSupportState(TypedDict):
    """State that flows through the routing workflow"""
    user_query: str                          # Original user question
    extracted_intent: Optional[str]          # Classified intent category
    external_service_response: Optional[Dict] # Raw response from external service
    final_response: str                      # LLM-generated personalized response
    
print("✅ State schema defined")

## 6. Node 1: Extract User Intent

This node uses an LLM to classify the user's support query into one of four intent categories: password_reset, hardware_issue, software_issue, or network_problem.

In [ ]:
def extract_user_intent_node(state: ITSupportState) -> Dict:
    """
    Step 1: Extract and classify user intent from support query
    """
    print("\n🔍 STEP 1: Extracting user intent...")
    
    user_query = state["user_query"]
    
    system_prompt = """You are an IT support intent classifier.
Analyze the user's query and classify it into ONE of these categories:
- password_reset: Password forgotten, locked account, login issues
- hardware_issue: Physical device problems (laptop, monitor, keyboard, etc.)
- software_issue: Application crashes, installation failures, software errors
- network_problem: Internet connectivity, WiFi issues, network access

Return ONLY the category name as a single word."""
    
    try:
        # Use LangChain ChatOpenAI with messages
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=user_query)
        ]
        
        # Use temperature=0 for classification consistency
        classification_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        response = classification_llm.invoke(messages)
        
        intent = response.content.strip().lower()
        print(f"   ✅ Intent classified: {intent}")
        
        return {"extracted_intent": intent}
        
    except Exception as e:
        print(f"   ❌ Error: {e}")
        return {"extracted_intent": "unknown"}

print("✅ Extract intent node defined")

## 7. Node 2: External Service - Password Reset

Simulates calling an external password reset service API. Returns structured data with verification steps, timing, and alternative methods.

In [ ]:
def call_password_reset_service(state: ITSupportState) -> Dict:
    """
    External Service: Password Reset API
    """
    print("\n🔑 EXTERNAL CALL: Password Reset Service...")
    
    # Simulate external API call
    service_response = SERVICE_DATA['password_reset']
    
    print(f"   ✅ Service responded: Priority {service_response['priority']}")
    print(f"   ⏱️  Estimated time: {service_response['estimated_time_minutes']} minutes")
    
    return {"external_service_response": service_response}

print("✅ Password reset service node defined")

## 8. Node 2: External Service - Hardware Issue

Simulates calling an external hardware support API. Returns ticket information, loaner device availability, and repair timelines.

In [ ]:
def call_hardware_service(state: ITSupportState) -> Dict:
    """
    External Service: Hardware Support API
    """
    print("\n🔧 EXTERNAL CALL: Hardware Support Service...")
    
    # Simulate external API call
    service_response = SERVICE_DATA['hardware_issue']
    
    print(f"   ✅ Ticket created: {service_response['ticket_id']}")
    print(f"   ⏱️  Estimated resolution: {service_response['estimated_resolution_days']} days")
    
    return {"external_service_response": service_response}

print("✅ Hardware service node defined")

## 9. Node 2: External Service - Software Issue

Simulates calling an external software support API. Returns troubleshooting steps, remote support options, and resolution estimates.

In [ ]:
def call_software_service(state: ITSupportState) -> Dict:
    """
    External Service: Software Support API
    """
    print("\n💻 EXTERNAL CALL: Software Support Service...")
    
    # Simulate external API call
    service_response = SERVICE_DATA['software_issue']
    
    print(f"   ✅ Ticket created: {service_response['ticket_id']}")
    print(f"   ⏱️  Estimated resolution: {service_response['estimated_resolution_minutes']} minutes")
    
    return {"external_service_response": service_response}

print("✅ Software service node defined")

## 10. Node 2: External Service - Network Problem

Simulates calling an external network support API. Returns diagnostic steps, network status, and temporary workarounds.

In [ ]:
def call_network_service(state: ITSupportState) -> Dict:
    """
    External Service: Network Support API
    """
    print("\n🌐 EXTERNAL CALL: Network Support Service...")
    
    # Simulate external API call
    service_response = SERVICE_DATA['network_problem']
    
    print(f"   ✅ Ticket created: {service_response['ticket_id']}")
    print(f"   ⏱️  Estimated resolution: {service_response['estimated_resolution_minutes']} minutes")
    
    return {"external_service_response": service_response}

print("✅ Network service node defined")

## 11. Routing Logic

Routes the workflow to the appropriate external service node based on the classified intent.

In [ ]:
def route_based_on_intent(state: ITSupportState) -> str:
    """
    Route to appropriate external service based on intent
    """
    intent = state.get("extracted_intent", "unknown")
    
    routing_map = {
        "password_reset": "password_reset_service",
        "hardware_issue": "hardware_service",
        "software_issue": "software_service",
        "network_problem": "network_service"
    }
    
    next_node = routing_map.get(intent, "password_reset_service")
    print(f"\n🔀 ROUTING: {intent} → {next_node}")
    
    return next_node

print("✅ Routing function defined")

## 12. Node 3: Generate Personalized Response

Uses an LLM to transform the raw external service response into a human-friendly, actionable message with clear steps and timing.

In [ ]:
def generate_personalized_response_node(state: ITSupportState) -> Dict:
    """
    Step 3: Generate personalized response from external service data
    """
    print("\n✨ STEP 3: Generating personalized response...")
    
    service_response = state["external_service_response"]
    intent = state["extracted_intent"]
    
    system_prompt = """You are a helpful IT support assistant.
Generate a clear, actionable response based on the external service data.

Format the response as:
**Answer**: [Brief identification of the issue]

**→ Immediate Action Available:**
  1. [First step]
  2. [Second step]
  3. [Third step]
  4. [Fourth step if applicable]

**Alternative:** [Alternative method with contact info]

**Estimated Resolution Time:** [time] (method)
**Priority:** [priority level] [additional context if relevant]

Keep it concise, professional, and actionable."""
    
    user_prompt = f"""Intent: {intent}
Service Response Data:
{json.dumps(service_response, indent=2)}

Generate a user-friendly IT support response."""
    
    try:
        # Use LangChain ChatOpenAI
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=user_prompt)
        ]
        
        response = llm.invoke(messages)
        final_response = response.content
        
        print(f"   ✅ Response generated ({len(final_response)} chars)")
        
        return {"final_response": final_response}
        
    except Exception as e:
        print(f"   ❌ Error: {e}")
        return {"final_response": "Your request has been received and is being processed."}

print("✅ Generate response node defined")

## 13. Build the Routing Graph

Construct the LangGraph workflow connecting the intent extraction, conditional routing to external services, and response generation.

In [ ]:
def create_routing_graph():
    """Build the routing workflow"""
    
    # Initialize graph builder
    builder = StateGraph(ITSupportState)
    
    # Add nodes
    builder.add_node("extract_intent", extract_user_intent_node)
    builder.add_node("password_reset_service", call_password_reset_service)
    builder.add_node("hardware_service", call_hardware_service)
    builder.add_node("software_service", call_software_service)
    builder.add_node("network_service", call_network_service)
    builder.add_node("generate_response", generate_personalized_response_node)
    
    # Define flow
    builder.add_edge(START, "extract_intent")
    
    # Conditional routing to external services
    builder.add_conditional_edges(
        "extract_intent",
        route_based_on_intent,
        {
            "password_reset_service": "password_reset_service",
            "hardware_service": "hardware_service",
            "software_service": "software_service",
            "network_service": "network_service"
        }
    )
    
    # All services converge to response generation
    builder.add_edge("password_reset_service", "generate_response")
    builder.add_edge("hardware_service", "generate_response")
    builder.add_edge("software_service", "generate_response")
    builder.add_edge("network_service", "generate_response")
    builder.add_edge("generate_response", END)
    
    # Compile graph
    graph = builder.compile()
    
    print("✅ Routing graph compiled")
    return graph

# Create the graph
graph = create_routing_graph()

## 14. Visualize the Workflow

Display the graph structure showing the routing flow with conditional branches to different external services.

In [ ]:
try:
    from IPython.display import Image, display
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print("📊 Graph Structure:")
    print("""
    ┌─────────────────┐
    │     START       │
    └────────┬────────┘
             ↓
    ┌─────────────────┐
    │ extract_intent  │  ← Step 1: Classify user query
    └────────┬────────┘
             ↓
       [Route Based on Intent]
      ┌──────┼──────┬──────┐
      ↓      ↓      ↓      ↓
    ┌────┐ ┌────┐ ┌────┐ ┌────┐
    │Pass│ │Hard│ │Soft│ │Net │  ← Step 2: External Service APIs
    │word│ │ware│ │ware│ │work│
    └──┬─┘ └──┬─┘ └──┬─┘ └──┬─┘
       └──────┼──────┼──────┘
              ↓
    ┌─────────────────┐
    │    generate     │  ← Step 3: Personalized response
    │    response     │
    └────────┬────────┘
             ↓
    ┌─────────────────┐
    │       END       │
    └─────────────────┘
    """)

## 15. Test the Routing Workflow

Run the complete workflow with different IT support queries to see the routing pattern in action.

In [ ]:
def run_it_support(query: str):
    """Execute the routing workflow with a user query"""
    print("="*70)
    print(f"💬 USER QUERY: {query}")
    print("="*70)
    
    try:
        # Invoke the graph
        result = graph.invoke({
            "user_query": query,
            "extracted_intent": None,
            "external_service_response": None,
            "final_response": ""
        })
        
        print("\n" + "="*70)
        print("🤖 ASSISTANT RESPONSE:")
        print("="*70)
        print(result["final_response"])
        print("\n")
        
        return result
        
    except Exception as e:
        print(f"\n❌ Error: {e}")
        return None

print("✅ Test function ready")

## 16. Test Case 1: Password Reset (Primary Scenario)

Test the exact scenario from the design image - urgent password reset with meeting deadline.

In [ ]:
run_it_support(
    "Hi IT Support, I forgot my password and I've been locked out of my account after 3 failed login attempts. I need to access it ASAP for an important meeting in 30 minutes. Please help!"
)

## 17. Test Case 2: Hardware Issue

Test routing to hardware support service with laptop failure scenario.

In [ ]:
run_it_support(
    "My laptop won't turn on at all. I tried charging it overnight but still nothing. I have urgent work to complete today."
)

## 18. Test Case 3: Software Issue

Test routing to software support service with application crash scenario.

In [ ]:
run_it_support(
    "Microsoft Office keeps crashing every time I try to open a Word document. I've tried restarting but the problem persists."
)

## 19. Test Case 4: Network Problem

Test routing to network support service with connectivity issue scenario.

In [ ]:
run_it_support(
    "I can't connect to the internet. WiFi shows connected but I can't access any websites or email. This started about an hour ago."
)

## 20. Interactive Testing Widget

Create an interactive interface to test the routing workflow with custom queries or predefined scenarios.

In [ ]:
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    
    # Create widgets
    query_input = widgets.Textarea(
        value='',
        placeholder='Enter your IT support query here...',
        description='Your Query:',
        layout=widgets.Layout(width='600px', height='100px'),
        style={'description_width': '100px'}
    )
    
    # Predefined test queries
    test_queries_dropdown = widgets.Dropdown(
        options=[
            ('-- Select a test query --', ''),
            ('Password Reset - Urgent', "Hi IT Support, I forgot my password and I've been locked out of my account after 3 failed login attempts. I need to access it ASAP for an important meeting in 30 minutes. Please help!"),
            ('Hardware - Laptop Failure', "My laptop won't turn on at all. I tried charging it overnight but still nothing. I have urgent work to complete today."),
            ('Software - Office Crash', "Microsoft Office keeps crashing every time I try to open a Word document. I've tried restarting but the problem persists."),
            ('Network - No Internet', "I can't connect to the internet. WiFi shows connected but I can't access any websites or email. This started about an hour ago."),
        ],
        description='Examples:',
        layout=widgets.Layout(width='600px'),
        style={'description_width': '100px'}
    )
    
    run_button = widgets.Button(
        description='Run Query',
        button_style='primary',
        icon='play'
    )
    
    output_area = widgets.Output()
    
    def on_dropdown_change(change):
        if change['new']:
            query_input.value = change['new']
    
    def on_run_click(button):
        with output_area:
            clear_output()
            if query_input.value.strip():
                run_it_support(query_input.value)
            else:
                print("⚠️  Please enter a query")
    
    test_queries_dropdown.observe(on_dropdown_change, names='value')
    run_button.on_click(on_run_click)
    
    # Display interface
    display(widgets.VBox([
        widgets.HTML("<h3>🧪 Interactive IT Support Routing Tester</h3>"),
        test_queries_dropdown,
        query_input,
        run_button,
        output_area
    ]))
    
except ImportError:
    print("⚠️  ipywidgets not available. Use run_it_support() function instead.")

## 21. Key Learnings: Routing Pattern

### ✅ **When to Use Routing Pattern**
- Requests that can be **categorized into distinct types**
- Each category requires **different specialized handling**
- Need to **integrate with multiple external services** or APIs
- Intent classification determines the execution path

### ⚠️ **When NOT to Use Routing Pattern**
- Tasks requiring **sequential multi-step processing**
- Workflows where **all requests follow the same path**
- Simple queries that don't need service differentiation
- Scenarios requiring **iterative loops or tool calling**

### 📊 **Pattern Comparison**

| Aspect | Routing | Prompt Chaining | Agentic |
|--------|---------|-----------------|----------|
| **Flow** | Branching paths | Linear sequence | Dynamic loops |
| **Decision Point** | At classification | At checkpoints | Continuous |
| **Service Calls** | One per request | Multiple sequential | Multiple iterative |
| **Complexity** | Low-Medium | Medium | High |
| **Use Case** | Intent-based routing | Multi-step tasks | Open-ended problems |

### 💡 **Best Practices**
1. **Clear intent categories** - Well-defined, mutually exclusive classifications
2. **Store raw service data** - Keep external responses unformatted in state
3. **Single service call** - Each route calls exactly one external service
4. **Converge to common endpoint** - All routes merge for response generation
5. **Handle unknown intents** - Provide fallback routing for edge cases

### 🎯 **This Implementation Demonstrated**
- ✅ LLM-based intent classification
- ✅ Conditional routing to specialized services
- ✅ External service simulation with realistic data
- ✅ Personalized response generation from raw service data
- ✅ Clean separation of concerns (classification → service → formatting)
- ✅ Production-ready error handling and logging

---

## 📚 Resources
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Workflows vs Agents](https://docs.langchain.com/oss/python/langgraph/workflows-agents)
- [Thinking in LangGraph](https://docs.langchain.com/oss/python/langgraph/thinking-in-langgraph)

---

*This notebook demonstrates a production-ready routing workflow for IT support automation with external service integration.*